In [80]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [2]:
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42'
}

In [3]:
data = pd.read_csv('result_allratings.csv', header=0, index_col=0)

In [22]:
data

,source_names,source_links,bias_ratings,community_feedbacks_agree,community_feedbacks_disagree
0,Andrew C. McCarthy,/news-source/andrew-c-mccarthy-media-bias,Right,79,66
1,Andrew Giuliani,/news-source/andrew-giuliani-media-bias,Lean Right,82,121
2,Andrew Napolitano,/news-source/andrew-napolitano,Right,1009,1189
3,Andrew Stuttaford,/news-source/andrew-stuttaford-media-bias,Lean Right,41,33
4,Andrew Sullivan,/news-source/andrew-sullivan-media-bias,Lean Right,333,361
...,...,...,...,...,...
1248,Yasmin Tavakoli,/news-source/yasmin-tavakoli-media-bias,Center,37,36
1249,Yes! Magazine,/news-source/yes-magazine,Left,492,265
1250,Zack Beauchamp,/news-source/zack-beauchamp-media-bias,Lean Left,24,30
1251,Zeeshan Aleem,/news-source/zeeshan-aleem-media-bias,Left,19,19


In [9]:
url = 'https://www.allsides.com' + data.source_links[1]

In [11]:
url = "https://www.allsides.com/news-source/abc-news-media-bias"

In [10]:
url

'https://www.allsides.com/news-source/andrew-giuliani-media-bias'

In [12]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [13]:
soup

<!DOCTYPE html>

<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">
<head profile="http://www.w3.org/1999/xhtml/vocab">
<meta charset="utf-8"/>
<link href="https://www.allsides.com/sites/default/files/AllSides-Icon.png" rel="shortcut icon" type="image/png"/>
<meta content="555035668344688" property="fb:app_id"/>
<meta content="Learn the media bias of ABC News. AllSides has rated the bias of hundreds of media outlets, sources and writers." name="description"/>
<link href="https://www.allsides.com/news-source/abc-news-media-bias" rel="canonical"/>
<link href="https://www.allsides.com/node/76" rel="shortlink"/>
<meta content="AllSides" property="og:site_name"/>
<meta content="article" property="og:type"/

In [16]:
float(soup.find('div',{'class':'numerical-bias-rating'}).text)

-2.4

In [28]:
r = requests.get('https://www.allsides.com/media-bias/ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B%5D=1&field_news_source_type_tid%5B%5D=2&field_news_source_type_tid%5B%5D=3&field_news_source_type_tid%5B%5D=4&field_news_source_type_tid%5B%5D=5&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=', headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [59]:
odd_url_first = soup.find('tr', {'class':'odd views-row-first'}).find('td', {'class':'views-field views-field-title source-title'}).a['href']
odd_name_first = soup.find('tr', {'class':'odd views-row-first'}).find('td', {'class':'views-field views-field-title source-title'}).text.strip()

In [58]:
even_url_list = [element.find('td', {'class':'views-field views-field-title source-title'}).a['href'] for element in soup.find_all('tr',{'class':'even'})]
odd_url_list = [element.find('td', {'class':'views-field views-field-title source-title'}).a['href'] for element in soup.find_all('tr',{'class':'odd'})]
even_url_last = soup.find('tr', {'class':'even views-row-last'}).find('td', {'class':'views-field views-field-title source-title'}).a['href']
even_name_list = [element.find('td', {'class':'views-field views-field-title source-title'}).text.strip() for element in soup.find_all('tr',{'class':'even'})]
odd_name_list = [element.find('td', {'class':'views-field views-field-title source-title'}).text.strip() for element in soup.find_all('tr',{'class':'odd'})]
even_name_last = soup.find('tr', {'class':'even views-row-last'}).find('td', {'class':'views-field views-field-title source-title'}).text.strip()

In [60]:
urls = [odd_url_first] + [x for pair in zip(even_url_list, odd_url_list) for x in pair] + [even_url_last]
names = [odd_name_first] + [x for pair in zip(even_name_list, odd_name_list) for x in pair] + [even_name_last]

In [61]:
names[1]

'"The Conversation" Contributor'

In [63]:
f'https://www.allsides.com/media-bias/ratings?page={page}&field_featured_bias_rating_value=All&field_news_source_type_tid%5B0%5D=1&field_news_source_type_tid%5B1%5D=2&field_news_source_type_tid%5B2%5D=3&field_news_source_type_tid%5B3%5D=4&field_news_source_type_tid%5B4%5D=5&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title='

'https://www.allsides.com/media-bias/ratings?page=1&field_featured_bias_rating_value=All&field_news_source_type_tid%5B0%5D=1&field_news_source_type_tid%5B1%5D=2&field_news_source_type_tid%5B2%5D=3&field_news_source_type_tid%5B3%5D=4&field_news_source_type_tid%5B4%5D=5&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title='

In [64]:
page = 1
urls_list = []
names_list = []
headers = {
    'authority': 'www.allsides.com',
    'method': 'GET',
    'scheme': 'https',
    'accept': 'XXXSCAESKXXX*/XXXSCAESKXXX*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cookie': 'ab-consent=236',
    'dnt': '1',
    'referer': 'https://www.allsides.com/media-bias/ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B%5D=1&field_news_source_type_tid%5B%5D=2&field_news_source_type_tid%5B%5D=3&field_news_source_type_tid%5B%5D=4&field_news_source_type_tid%5B%5D=5&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=',
    'sec-ch-ua': '"Chromium";v="112", "Microsoft Edge";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': 'macOS',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48',
    'x-requested-with': 'XMLHttpRequest'
}
while True:
    try:
        r = requests.get(f'https://www.allsides.com/media-bias/ratings?page={page}&field_featured_bias_rating_value=All&field_news_source_type_tid%5B0%5D=1&field_news_source_type_tid%5B1%5D=2&field_news_source_type_tid%5B2%5D=3&field_news_source_type_tid%5B3%5D=4&field_news_source_type_tid%5B4%5D=5&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=', headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        odd_url_first = soup.find('tr', {'class':'odd views-row-first'}).find('td', {'class':'views-field views-field-title source-title'}).a['href']
        odd_name_first = soup.find('tr', {'class':'odd views-row-first'}).find('td', {'class':'views-field views-field-title source-title'}).text.strip()
        even_url_list = [element.find('td', {'class':'views-field views-field-title source-title'}).a['href'] for element in soup.find_all('tr',{'class':'even'})]
        odd_url_list = [element.find('td', {'class':'views-field views-field-title source-title'}).a['href'] for element in soup.find_all('tr',{'class':'odd'})]
        even_url_last = soup.find('tr', {'class':'even views-row-last'}).find('td', {'class':'views-field views-field-title source-title'}).a['href']
        even_name_list = [element.find('td', {'class':'views-field views-field-title source-title'}).text.strip() for element in soup.find_all('tr',{'class':'even'})]
        odd_name_list = [element.find('td', {'class':'views-field views-field-title source-title'}).text.strip() for element in soup.find_all('tr',{'class':'odd'})]
        even_name_last = soup.find('tr', {'class':'even views-row-last'}).find('td', {'class':'views-field views-field-title source-title'}).text.strip()
        urls = [odd_url_first] + [x for pair in zip(even_url_list, odd_url_list) for x in pair] + [even_url_last]
        names = [odd_name_first] + [x for pair in zip(even_name_list, odd_name_list) for x in pair] + [even_name_last]
        urls_list += urls
        names_list += names
        page += 1
    except Exception as e:
        print(page, e)
        continue

33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'
33 'NoneType' object has no attribute 'find'


KeyboardInterrupt: 

In [67]:
data = pd.DataFrame({'name':names_list, 'url':urls_list, 'lean': '', 'score': ''})

In [69]:
data

,name,url,lean,score
0,American Press,/news-source/american-press,,
1,American Psychological Association,/news-source/american-psychological-associatio...,,
2,American Press,/news-source/american-press,,
3,American Values Coalition,/news-source/american-values-coalition-media-bias,,
4,American Thinker,/news-source/american-thinker,,
...,...,...,...,...
1613,Zack Beauchamp,/news-source/zack-beauchamp-media-bias,,
1614,Yuma Sun,/news-source/yuma-sun-media-bias,,
1615,ZeroHedge,/news-source/zerohedge-media-bias,,
1616,Zeeshan Aleem,/news-source/zeeshan-aleem-media-bias,,


In [70]:
data.url[0]

'/news-source/american-press'

In [87]:
def get_numerical_bias_rating(soup):
    try:
        bias_rating = float(soup.find('div', {'class': 'numerical-bias-rating'}).text)
    except Exception:
        bias_rating = None
    return bias_rating

In [132]:
headers = {
    'authority': 'www.allsides.com',
    'method': 'GET',
    'scheme': 'https',
    'accept': 'XXXSCAESKXXX*/XXXSCAESKXXX*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'cookie': 'ab-consent=236',
    'dnt': '1',
    'sec-ch-ua': '"Chromium";v="112", "Microsoft Edge";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': 'macOS',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48',
    'x-requested-with': 'XMLHttpRequest'
}
for i in tqdm(range(1583, data.shape[0])):
    r = requests.get('https://www.allsides.com/'+data.url[i], headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    data.lean[i] = soup.find('span', {'class': 'bias-rating'}).text
    data.score[i] = get_numerical_bias_rating(soup)

100%|██████████| 35/35 [01:32<00:00,  2.64s/it]


In [135]:
data.to_csv('result_allrating_continuous_original.csv', index=False)

In [137]:
data

,name,url,lean,score
0,American Press,/news-source/american-press,Lean Left,None
1,American Psychological Association,/news-source/american-psychological-associatio...,Center,None
2,American Press,/news-source/american-press,Lean Left,None
3,American Values Coalition,/news-source/american-values-coalition-media-bias,Center,None
4,American Thinker,/news-source/american-thinker,Right,4.0
...,...,...,...,...
1613,Zack Beauchamp,/news-source/zack-beauchamp-media-bias,Lean Left,None
1614,Yuma Sun,/news-source/yuma-sun-media-bias,Center,None
1615,ZeroHedge,/news-source/zerohedge-media-bias,Lean Right,2.0
1616,Zeeshan Aleem,/news-source/zeeshan-aleem-media-bias,Left,None


In [144]:
data = data[data.lean!='Mixed'].reset_index(drop=True)

In [145]:
data.lean.value_counts()

Center        730
Lean Left     291
Left          202
Lean Right    192
Right         185
Name: lean, dtype: int64

In [146]:
data.score

0       None
1       None
2       None
3       None
4        4.0
        ... 
1595    None
1596    None
1597     2.0
1598    None
1599     2.0
Name: score, Length: 1600, dtype: object

In [149]:
data[['lean','score']]

,lean,score
0,Lean Left,None
1,Center,None
2,Lean Left,None
3,Center,None
4,Right,4.0
...,...,...
1595,Lean Left,None
1596,Center,None
1597,Lean Right,2.0
1598,Left,None


In [152]:
for i in tqdm(range(data.shape[0])):
    if data.score[i] == None:
        if data.lean[i] == 'Left':
            data.score[i] = -4.5
        elif data.lean[i] == 'Lean Left':
            data.score[i] = -2
        elif data.lean[i] == 'Center':
            data.score[i] = 0
        elif data.lean[i] == 'Lean Right':
            data.score[i] = 2
        elif data.lean[i] == 'Right':
            data.score[i] = 4.5

100%|██████████| 1600/1600 [00:00<00:00, 17470.53it/s]


In [153]:
data.score.any(None)

/var/folders/tv/_hp7fqz10tlgb38nnwr3p4bm0000gn/T/ipykernel_9308/1074488502.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data.score.any(None)


True

In [158]:
data.score

0        -2
1         0
2        -2
3         0
4       4.0
       ... 
1595     -2
1596      0
1597    2.0
1598   -4.5
1599    2.0
Name: score, Length: 1600, dtype: object

In [161]:
data.to_csv('result_allrating_continuous_washed.csv', index=False)